# TOPIC NAME

## Introduction

This guide will show you how to:

* TODO
* TODO

By the end of it, you will TODO!

## Setup

Install dependencies

In [ ]:
pip install neptune-client TODO

## Step 1: TODO

TODO

In [ ]:
import dalex as dx
import neptune
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from neptunecontrib.api import log_explainer, log_local_explanations, log_global_explanations

project = neptune.init(project_qualified_name='shared/dalex-integration',
                       api_token='ANONYMOUS')

data = dx.datasets.load_titanic()

X = data.drop(columns='survived')
y = data.survived

numerical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['age', 'fare', 'sibsp', 'parch']),
        ('cat', categorical_transformer, ['gender', 'class', 'embarked'])
    ]
)

classifier = MLPClassifier(hidden_layer_sizes=(100, 200, 20), max_iter=400, random_state=0)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])

neptune.create_experiment('explanations')

clf.fit(X, y)

new_observation = pd.DataFrame({'gender': ['male'],
                                'age': [25],
                                'class': ['1st'],
                                'embarked': ['Southampton'],
                                'fare': [72],
                                'sibsp': [0],
                                'parch': 0},
                               index=['John'])

expl = dx.Explainer(clf, X, y, label="Titanic MLP Pipeline")

log_local_explanations(expl, new_observation)
log_global_explanations(expl, categorical_features=["gender", "class"], numerical_features=["age", "fare"])
log_explainer('explainer.pkl', expl)

## Step 2: Initialize Neptune

Connects your script to Neptune application. 

In [ ]:
import neptune

neptune.init(
    api_token="ANONYMOUS",
    project_qualified_name="shared/onboarding"
)

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune project `project_qualified_name`.

---

**Note:** 


Instead of logging data to the public project 'shared/onboarding' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your username to the ``project_qualified_name`` argument of the ``neptune.init()`` method: ``project_qualified_name='YOUR_USERNAME/sandbox``. Keep `/sandbox` at the end, the `sandbox` project that was automatically created for you.

For example:

```python
neptune.init(project_qualified_name='funky_steve/sandbox', 
             api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn',
            )
```

---

## Step 3: Create an experiment

In [ ]:
neptune.create_experiment(name='great-idea')

This opens a new "experiment" namespace in Neptune to which you can log various objects.

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

## Step 4: TODO

TODO

In [ ]:
TODO

---

**Note:**
   
When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running `neptune.stop()`.

If you are running Neptune in regular `.py` scripts it will stop automatically when your code stops running.

---

In [ ]:
neptune.stop()